## Hazard assessment methodology
The impact of the single hazard (e.g. extreme precipitation, heat waves) can be evaluated on the different assets of the infrastructure, for example on the terminal, runway, taxiways, parking areas etc.

The idea is that this script can be run by expert users to understand and potentially modify how the hazard map data is calculated for use in the risk toolbox CLIMAAX.  
Describe where the data can be found, is there an API to download it or the files can be downloaded from some data repository. Provide a link to the repository (as DOI if possible).

The data used in this section can be found on Copernicu Climate Change Data Store (C3S) and they are related to UERRA SURFACE-MESCAN (https://cds.climate.copernicus.eu/datasets/reanalysis-uerra-europe-single-levels?tab=download)
on single level. The automathic dowload was made through Copernicus API (cdsapi python packages) both for temperature and precipitation data. 

The UERRA dataset was chosen for it's horizontal and temporal resolutio, providing a quite high resolution with continuos measurment. On the opposite, the 5.5km horizontal resolution could be to course for airport hazard assessment in order to evaluate this type of indicator for single airport parts (e.g taxiways and runway)

### Load libraries
xclim is a Python library tailored for climate services, offering a wide range of tools to compute climate-related indicators. In this notebook, it is the primary library used to calculate the climate indicator "number of days above a threshold temperature".

In [ ]:
from useful_functions import get_subfiles

import os
import xarray as xr
import xclim

### Create the directory structure

In [ ]:
nc_files = "../data_conv"
general_path = '../indicators/uerra'

## Process the data
This script processes the temperature and precipitation files from UERRA within the folder 'nc_files' and performs the calculation of some indicators. Specifically:

- Avoids recomputation by checking if output files already exist.
- Calculates the number of days exceeding specific temperature thresholds (35, 40 and 45 Celcius).
- Calculates the percentiles for both temperature and precipitation. The code uses xarray for handling NetCDF files, and xclim to compute climate indices like the number of hot days and percentiles. For each file, the results are saved as new NetCDF files in a specified directory.
- it handles missing values to avoid errors in the calculations.

In [ ]:
# Loop through all files in the folder
for file, file_path in get_subfiles(nc_files):
    if file.endswith(".nc"):  # Check if file is .nc
        # Load the dataset with lazy loading and chunking
        ds = xr.open_dataset(file_path) #, chunks={'time': 'auto'})

        try:
            print(file)
            if "t2m" in ds.data_vars:
                print('Temperature')
                # Compute maximum daily temperature and convert to Celsius
                dailyMaxTemp = ds['t2m'].resample(time='D').max()
                dailyMaxTemp.attrs['units'] = 'C'  # Set the units

                # -------- Define the output files for Number of Days Above Thresholds  --------
                output_files_numDays = {
                    'NumbDaysAbove30.nc': "30 C",
                    'NumbDaysAbove35.nc': "35 C",
                    'NumbDaysAbove40.nc': "40 C",
                    'NumbDaysAbove45.nc': "45 C",
                }

                for fname, threshold in output_files_numDays.items():
                    output_path = os.path.join(general_path, fname)

                    if os.path.exists(output_path):
                        print(f"The file {fname} already exists. Skipping calculation.")
                    else:
                        # Compute the number of days above the threshold
                        with xclim.set_options(cf_compliance="log"):
                            NumbDaysAbove = xclim.atmos.tx_days_above(tasmax=dailyMaxTemp, thresh=threshold, freq="YS")
                            NumbDaysAbove_avg = NumbDaysAbove.mean(dim='time', skipna=True)

                            # Save it
                            NumbDaysAbove_avg.to_netcdf(output_path)
                            print(f"Saved {fname} to {output_path}.")

                # -------- Define the output files for Percentiles ------------
                output_files_percent = {
                    'Temp_P95.nc': 0.95,
                    'Temp_P999.nc': 0.999,
                }

                for fname_percent, percentile in output_files_percent.items():
                    print(fname_percent, percentile)
                    output_path_percent = os.path.join(general_path, fname_percent)

                    if os.path.exists(output_path_percent):
                        print(f"The file {fname_percent} already exists. Skipping calculation.")
                    else:
                        # Calculate the percentiles across all time steps
                        dailyMaxTemp_nonan = dailyMaxTemp.dropna(dim='time', how='all')
                        calc_percentile = dailyMaxTemp_nonan.quantile(percentile, dim='time')

                        # Save it
                        calc_percentile.to_netcdf(output_path_percent)
                        print(f"Saved {percentile * 100}th percentile to {output_path_percent}.")

            elif "tp" in ds.data_vars:
                dailyTotPrep = ds['tp']
                # -------- Define the output files for Percentiles ------------
                output_files_percent_prec = {
                    'Precip_P95.nc': 0.99,
                    'Precip_P995.nc': 0.995,
                    'Precip_P999.nc': 0.999}

                for fname_percent_prep, percentile_prep in output_files_percent_prec.items():
                    print(fname_percent_prep, percentile_prep)
                    output_path_percent_prep = os.path.join(general_path, fname_percent_prep)

                    if os.path.exists(output_path_percent_prep):
                        print(f"The file {fname_percent_prep} already exists. Skipping calculation.")
                    else:
                        # Calculate the percentiles across all time steps
                        dailyTotPrep_nonan = dailyTotPrep.dropna(dim='time', how='all')
                        calc_percentile_prep = dailyTotPrep_nonan.quantile(percentile_prep, dim='time')

                        # Save it
                        calc_percentile_prep.to_netcdf(output_path_percent_prep)
                        print(f"Saved {percentile_prep * 100}th percentile to {output_path_percent_prep}.")
        finally:
            # close the dataset to free memory
            ds.close()
